In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


##k-fold cross validation

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
from sklearn.model_selection import KFold
import torch.nn as nn
import torch.optim as optim

# Definir transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Ajustar el tamaño
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza los valores de pix
])

# Cargar el conjunto de datos
dataset = ImageFolder(root='/Users/luisgc/Desktop/E/SP/luis', transform=transform)

# Verifica que las clases están correctamente identificadas
print(f"Clases encontradas: {dataset.classes}")

'''Modelo para prueba'''

# Modelo simple (CNN) para clasificación de las tres clases
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 64 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, 3)  # 3 clases: vaca_de_pie, vaca_acostada, cama_vacia
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layer(x)
        return x

''' k-fold cross validation '''

# Definir el número de folds
k_folds = 5

# Definir KFold para dividir los datos
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Diccionario para almacenar los resultados de cada fold
results = {}

# Hiperparametros
num_epochs = 5
learning_rate = 0.001

# Ciclo para los folds
for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):

    print(f'Fold {fold+1}/{k_folds}')

    # Crear subsets para este fold
    train_subset = Subset(dataset, train_idx)
    test_subset = Subset(dataset, test_idx)

    # DataLoader para entrenamiento y validación
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

    # Inicializar el modelo, criterio (loss) y optimizador
    model = SimpleCNN()
    criterion = nn.CrossEntropyLoss()  # Para clasificación multiclase
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Entrenamiento por epocas
    for epoch in range(num_epochs):
        model.train()  # Modo de entrenamiento
        running_loss = 0.0

        for images, labels in train_loader:
            optimizer.zero_grad()  # Resetear gradientes
            outputs = model(images)  # Forward
            loss = criterion(outputs, labels)  # Calcular pérdida
            loss.backward()  # Backpropagation
            optimizer.step()  # Actualizar pesos

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Evaluación en el conjunto de validacion
    model.eval()  # Modo de evaluacion
    correct = 0
    total = 0

    with torch.no_grad():  # No se calculan gradientes en evaluación
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Predicción
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Fold {fold+1} Accuracy: {accuracy}%')

    # Guardar los resultados de este fold
    results[fold] = accuracy

# Resultados finales
print("\nFinal Cross-Validation Results:")
for fold in range(k_folds):
    print(f"Fold {fold+1}: {results[fold]}%")
print(f'Average Accuracy: {sum(results.values())/len(results)}%')

##Split en carpetas

In [ ]:
import os
import shutil
import numpy as np

# Configuraciones de directorios
base_dir = '/Users/luisgc/Desktop/dataset_vacas'
train_dir = '/Users/luisgc/Desktop/train'
val_dir = '/Users/luisgc/Desktop/validation'
test_dir = '/Users/luisgc/Desktop/test'

# Crear directorios de salida
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = ['vaca_de_pie', 'vaca_acostada', 'cama_vacia']

# Separar datos
for class_name in classes:
    class_path = os.path.join(base_dir, class_name)
    images = os.listdir(class_path)
    np.random.shuffle(images)  # Mezclar las imágenes

    # Calcular los tamaños de los conjuntos
    n_total = len(images)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.15)

    # Separar las imágenes
    train_images = images[:n_train]
    val_images = images[n_train:n_train + n_val]
    test_images = images[n_train + n_val:]

    # Crear las carpetas para cada conjunto
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Mover las imágenes a sus respectivas carpetas
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))

# Imprimir el total de imágenes en cada carpeta
def print_image_counts(base_dir, classes):
    for class_name in classes:
        train_count = len(os.listdir(os.path.join(train_dir, class_name)))
        val_count = len(os.listdir(os.path.join(val_dir, class_name)))
        test_count = len(os.listdir(os.path.join(test_dir, class_name)))
        print(f"Clase: {class_name}")
        print(f"  Entrenamiento: {train_count}")
        print(f"  Validación: {val_count}")
        print(f"  Prueba: {test_count}")
        print("")

print_image_counts(base_dir, classes)